[Build an agent](https://python.langchain.com/docs/tutorials/agents/)

# End to end agent

In [1]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model = ChatOpenAI(model_name="gpt-4o-mini")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in madrid, spain")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in madrid, spain
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How can I assist you today?


In [3]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_efbP0Ou7p3IeFunidiT9rLS4)
 Call ID: call_efbP0Ou7p3IeFunidiT9rLS4
  Args:
    query: Madrid Spain weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Madrid', 'region': 'Madrid', 'country': 'Spain', 'lat': 40.4, 'lon': -3.6833, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1741546972, 'localtime': '2025-03-09 20:02'}, 'current': {'last_updated_epoch': 1741546800, 'last_updated': '2025-03-09 20:00', 'temp_c': 7.4, 'temp_f': 45.3, 'is_day': 0, 'condition': {'text': 'Moderate rain', 'icon': '//cdn.weatherapi.com/weather/64x64/night/302.png', 'code': 1189}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 327,

In [4]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the time where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the time where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_gJbx5dkMzolJGLkSGCpjxoYG)
 Call ID: call_gJbx5dkMzolJGLkSGCpjxoYG
  Args:
    query: current time in Madrid Spain
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://time.is/Madrid", "content": "Madrid: 07:11pm Time.is Time in Madrid, Spain now Sun: ↑ 08:09AM ↓ 06:49PM (10h 40m) - More info - Make Madrid time default - Add to favorite locations Time zone info for Madrid The time in Madrid is normally 6 hours ahead of the time in United States, but because these time zones don't share the same start and end times for daylight saving time, the time in Madrid can for a short while be 5 hours ahead of the time in United States. The IANA time zone identifier for Madrid is Euro

# Step-by-step guide to build an agent

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741546632, 'localtime': '2025-03-09 11:57'}, 'current': {'last_updated_epoch': 1741545900, 'last_updated': '2025-03-09 11:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 197, 'wind_dir': 'SSW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 50, 'feelslike_c': 13.0, 'feelslike_f': 55.4, 'windchill_c': 10.2, 'windchill_f': 50.3, 'heatindex_c': 10.5, 'heatindex_f': 50.8, 'dewpoint_c': 8.3, 'dewpoint_f': 46.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.9, 'gust_mph': 3.7, 'gust_kph': 6.0}}"}, {'url': 'https://weather.com/weather/today/l/Sa

In [6]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [8]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_SOlVXerZtVurEbpnFtXKgHDc', 'type': 'tool_call'}]


In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='f7a24fd6-a90b-4cf9-becd-d6413c468faf'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e16d63a4-9113-4a32-9d8e-d41552f88783-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='f5b7feba-cb25-4b73-8898-ffa1f5dad91e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xHyrw5G5NRV1QhyeFDP2AyG6', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8067a7ee-2c46-4e6f-9acb-dd29efd32eac-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_xHyrw5G5NRV1QhyeFDP2AyG6', 'ty

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_XvaV0QRj2Nv7gC48cPIV7I1U)
 Call ID: call_XvaV0QRj2Nv7gC48cPIV7I1U
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741546805, 'localtime': '2025-03-09 12:00'}, 'current': {'last_updated_epoch': 1741545900, 'last_updated': '2025-03-09 11:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.9, 'wi

In [15]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| as| follows|:

|-| **|Temperature|**|:| |12|.|8|°C| (|55|°F|)
|-| **|Condition|**|:| Part|ly| cloudy|
|-| **|Wind|**|:| |2|.|9| mph| (|4|.|7| k|ph|)| from| the| south|-s|outh|west|
|-| **|Humidity|**|:| |64|%
|-| **|Visibility|**|:| |16| km| (|9| miles|)

|You| can| find| more| details| [|here|](|https|://|www|.weather|api|.com|/|).|The| current| weather| in| San| Francisco|,| CA| is| as| follows|:

|-| **|Temperature|**|:| |12|.|8|°C| (|55|.|0|°F|)
|-| **|Condition|**|:| Part|ly| cloudy|
|-| **|Wind|**|:| |2|.|9| mph| (|4|.|7| k|ph|)| from| the| S|SW|
|-| **|Humidity|**|:| |64|%
|-| **|Cloud| Coverage|**|:| |50|%
|-| **|Visibility|**|:| |16| km| (|9| miles|)
|-| **|Last| Updated|**|:| March| |9|,| |202|5|,| at| |11|:|45| AM| local| time|

|For| more| detailed| updates|,| you| can| check| the| weather| on| [|Weather| API|](|https|://|www|.weather|api|.com|/)| or| [|The| Weather| Channel|](|https|://|weather|.com|/weather|/t|oday|/l|/|US|CA

KeyboardInterrupt: 

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 83, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e541ff27-8d71-4ba5-8dfa-a09b6b8756fa-0', usage_metadata={'input_tokens': 83, 'output_tokens': 12, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 106, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-85e960e5-da40-4eed-bdb5-962351c7cece-0', usage_metadata={'input_tokens': 106, 'output_tokens': 16, 'total_tokens': 122, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [20]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't know your name. If you'd like to share it, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 84, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-add75bdc-364c-49b8-a0f6-0efb31a50854-0', usage_metadata={'input_tokens': 84, 'output_tokens': 18, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
